In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import integrate
from scipy.integrate import solve_ivp

In [2]:
plt.rc('text', usetex = True)
plt.rc('font', **{'family' : "sans-serif"})
params = {'text.latex.preamble' : [r'\usepackage{amsmath}']}
plt.rcParams.update(params)
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

In [3]:
# Sistema de ecuaciones

def funciones(xi, y, sigma, n, h):
    psi_, eta_, psi1_, eta1_ = y
    dydxi = [h*(eta_ + sigma*xi**(3)*psi_**(n+1))*(1 + sigma*psi_)/xi/(2*(n+1)*sigma*eta_ - xi) 
           ,xi**(2)*psi_**(n) 
           ,h*(((eta1_ + 3*sigma*xi**(2)*psi_**(n+1) + sigma*xi**(3)*(n+1)*psi_**(n)*psi1_)*(sigma*psi_ + 1)/xi/(2*(n+1)*sigma*eta_ - xi)) + (sigma*psi1_*(eta_ + sigma*xi**(3)*psi_**(n+1))/xi/(2*(n+1)*sigma*eta_ - xi)) - ((1 + sigma*psi_)*(eta_ + sigma*xi**(3)*psi_**(n+1))/xi**(2)/(2*(n+1)*sigma*eta_ - xi)) - ((1 + sigma*psi_)*(eta_ + sigma*xi**(3)*psi_**(n+1))*(2*(n+1)*sigma*eta1_ - 1)/xi/((2*(n+1)*sigma*eta_ - xi)**(2))))           
           ,2*xi*psi_**(n) + xi**(2)*n*psi_**(n-1)*psi1_]
    return dydxi

In [4]:
c = 2.997*10**(8)               # Velocidad de la luz en S.I.
G = 6.673*10**(-11)             # Constante Gravitación Universal en S.I. 
Msun = 1.989*10**(30)           # 1 Masa solar en S.I.

In [5]:
                                               ## Lista de parámetros ##

# C
C_inicial = 0                    # C inicial            
C_final =  1/4                   # C final
Paso_C = 1/32                    # Diferencia entre valores consecutivos
Num_C = round((Paso_C + C_final - C_inicial)/Paso_C)  # Número de valores
Lista_C = np.linspace(C_inicial,C_final,Num_C)        # Lista con valores desde C inicial hasta C final
print('C interval: ',end='')
print(*Lista_C, sep=', ')

# n 
n_inicial = 0.5          # n inicial 
n_final = 4.0            # n final
Paso_n = 0.5             # Diferencia entre valores consecutivos
Num_n = round((Paso_n + n_final - n_inicial)/Paso_n)  # Número de valores
Lista_n = np.linspace(n_inicial,n_final,Num_n)        # Lista con valores desde n inical hasta n final
Lista_n = [0.5,0.999999,1.5,2.0,2.5,3.0,3.5,4.0]
print('n interval: ',end='')                                                    
print(*Lista_n, sep=', ')

# Sigma = razón entre Presión central y Densidad central
sigma_inicial = 0.05             # sigma inicial             
sigma_final = 0.8                # sigma final
Paso_sigma = 0.05                # Diferencia entre valores consecutivos
Num_sigma = round((Paso_sigma + sigma_final - sigma_inicial)/Paso_sigma)  # Número de valores
Lista_sigma = np.linspace(sigma_inicial,sigma_final,Num_sigma)            # Lista con valores desde sigma inical hasta sigma final
print('\u03c3 interval: ',end='')
print(*Lista_sigma, sep=', ')

Number_of_models = len(Lista_C)*len(Lista_n)*len(Lista_sigma)
print('Number of models: ', Number_of_models)

rho_c = 2.5*10**(18)               # Densidad central

C interval: 0.0, 0.03125, 0.0625, 0.09375, 0.125, 0.15625, 0.1875, 0.21875, 0.25
n interval: 0.5, 0.999999, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0
σ interval: 0.05, 0.1, 0.15000000000000002, 0.2, 0.25, 0.3, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.6500000000000001, 0.7000000000000001, 0.7500000000000001, 0.8
Number of models:  1152


In [6]:
ListaModelos = []                                        # Container for all models 

for i in range(len(Lista_C)):
    
    ListaModelos.append([])                              # Container for C
    
    for j in range(len(Lista_n)):
        
        ListaModelos[i].append([])                       # Container for n

In [7]:
%%time

for i in range(len(Lista_C)):
    
    C = Lista_C[i]
    h = 1 - 2*C
    
    for j in range(len(Lista_n)):
        
        n = Lista_n[j]
        
        for k in range(len(Lista_sigma)):
            
            sigma = Lista_sigma[k]
                    
            K = sigma*c**(2)/rho_c**(1/n)      # k**(n) en m**(3)/kg
                     
            # Condiciones iniciales
            Psi0 = 1.0
            Eta0 = 0.0
            Psi10 = 0.0
            Eta10 = 0.0
            
            y0 = [Psi0,Eta0,Psi10,Eta10]
            
            def stop_condition(xi,y,sigma,n,h):
                return y[0] - 10**-15
            
            stop_condition.terminal = True
            
            xi0 = 10**(-15)
            ximax = 10000
    
            xi_span = (xi0,ximax)
        
            soluciones = solve_ivp(funciones,xi_span,y0,method='RK45',events=stop_condition,
                                   args=(sigma,n,h),max_step=1/50)
        
            
            Psi = soluciones.y[0]
            Eta = soluciones.y[1]
            Psi1 = soluciones.y[2]
            Eta1 = soluciones.y[3]
            xi = soluciones.t 
                
            #########################################################################################################
            
            xiNorm = xi/xi[-1]                                       # xi normalized
            
            m = (c**(2)*sigma*(n+1)/(4*np.pi*rho_c)**(1/3)/G)**(3/2)*Eta/Msun   # Masa en unidades de masa solar
            r = (c**(2)*sigma*(n+1)/4/np.pi/G/rho_c)**(1/2)*xi/1000             # Radio en kilómetros
            mr = 2*sigma*(n+1)*Eta/xi                                           # 2*G*m/c**(2)/r
            
            Densidad = Psi**(n)                                      # Density
            
            Dprima = n*Psi**(n-1)*Psi1                               # Density gradient
            
            # Pressure divided by central pressure
            PNorm = Psi**(n+1)
            
            # Tangential pressure
            PTNorm = (C*(n+1)*(Eta + sigma*Psi**(n+1)*xi**(3))/(xi - 2*Eta*sigma*(n+1)))*(Psi**(n) + sigma*Psi**(n+1)) + Psi**(n+1)
            
                        
            Pprima = (n+1)*Psi**(n)*Psi1
            
            # Tangential pressure gradient
            Ptprima = C*(n+1)*((Eta1 + 3*sigma*xi**(2)*Psi**(n+1) + sigma*(n+1)*xi**(3)*Psi**(n)*Psi1)*(Psi**(n) + sigma*Psi**(n+1))/(xi - 2*Eta*(n+1)*sigma) + (Eta + sigma*xi**(3)*Psi**(n+1))*(n*Psi**(n-1)*Psi1 + sigma*(n+1)*Psi**(n)*Psi1)/(xi - 2*Eta*(n+1)*sigma) - (Eta + sigma*xi**(3)*Psi**(n+1))*(Psi**(n) + sigma*Psi**(n+1))*(1 - 2*Eta1*(n+1)*sigma)/(xi - 2*Eta*(n+1)*sigma)**(2)) + (n+1)*Psi**(n)*Psi1
            
            SEC = (1/sigma)*Densidad - PNorm - 2*PTNorm     # Strong energy condition (SEC)   
            
            v2r =  sigma*(1 + 1/n)*Psi                      # Sound speed squared
            
            # Tangential sound speed squared
            v2t = sigma*(1 + 1/n)*Psi +  C*sigma*(n+1)*(Eta + sigma*Psi**(n+1)*xi**(3))/(xi - 2*Eta*sigma*(n+1))
            
            Gamma = v2r*(Densidad + sigma*PNorm)/PNorm/sigma
            
            # Second derivative of Psi
            Psi11 = h*(((Eta1 + 3*sigma*xi**(2)*Psi**(n+1) + sigma*xi**(3)*(n+1)*Psi**(n)*Psi1)*(sigma*Psi + 1)/xi/(2*(n+1)*sigma*Eta - xi)) + (sigma*Psi1*(Eta + sigma*xi**(3)*Psi**(n+1))/xi/(2*(n+1)*sigma*Eta - xi)) - ((1 + sigma*Psi)*(Eta + sigma*xi**(3)*Psi**(n+1))/xi**(2)/(2*(n+1)*sigma*Eta - xi)) - ((1 + sigma*Psi)*(Eta + sigma*xi**(3)*Psi**(n+1))*(2*(n+1)*sigma*Eta1 - 1)/xi/((2*(n+1)*sigma*Eta - xi)**(2))))
            
            # Cracking: deltaR1: Density, deltaR2: Mass, deltaR3: Pressure, delta R4: Pressure gradient   
            deltaR1 = h*((n+1) * sigma * (Eta + xi**(3) * sigma * Psi**(n+1)) / xi / (xi - 2 * sigma * n * Eta - 2 * sigma * Eta))
            deltaR2 = h*(Psi**(n+1) * xi**(2) * sigma * (n+1) * (1 + sigma * Psi) * (1 + 2 * xi**(2) * sigma * sigma * (n+1) * Psi**(n+1)) / n / Psi1 / ((xi - 2 * Eta * sigma * n - 2 * Eta * sigma)**(2)))
            deltaR3 = h*(sigma * (n+1)**(2) * sigma * Psi * (Eta + xi**(3) * Psi**(n) + 2 * sigma * xi**(3) * Psi**(n+1)) / n / xi / (xi - 2 * sigma * n * Eta - 2 * sigma * Eta))
            deltaR4 = sigma * (n+1) * (n * Psi1**(2) + Psi * Psi11) / n / Psi1
            deltaRPolitropa = deltaR1 + deltaR2 + deltaR3 + deltaR4
            
            # Buoyancy
            Psin11 = n*(Psi**(n-1)*Psi11 + (n-1)*Psi**(n-2)*Psi1**(2))
            
            M = (c**(2)*(n+1)/(4*np.pi)**(1/3)/G)**(3/2)*(k/c**(2))**(n/2)*sigma**((3-n)/2)*Eta[-1]/Msun  # Masa en unidades de masa solar
            
            R = (c**(2)*(n+1)/4/np.pi/G)**(1/2)*(k/c**(2))**(n/2)*sigma**((1-n)/2)*xi[-1]/1000         # Radio en kilómetros
            
            ##########################################################################################################
            
                                         #0    #1     #2       #3     #4      #5      #6      #7                       
            ListaModelos[i][j].append([xiNorm, mr, Densidad, PNorm, PTNorm, Dprima, Pprima, Ptprima,
                                        #8  #9   #10   #11          #12         #13
                                       SEC, v2r, v2t, Gamma, deltaRPolitropa, Psin11])

C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\DANIEL\Anaconda3\lib\site-packages\ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in add


KeyboardInterrupt: 

# <center> 2m/r < 1
# <center> C1, C2 and C3

In [8]:
count = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if any(ListaModelos[i][j][k][1] >= 1):
                
                print('2m/r > 1. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count = count + 1
                
            else:
                continue
                
print('Número total de modelos que no cumplen 2m/r < 1:           ', count, ' de ', Number_of_models)


IndexError: list index out of range

# <center> Density, Radial Pressure and Tangential Pressure   > 0
# <center> C4

In [ ]:
count1 = 0
count2 = 0
count3 = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if any(ListaModelos[i][j][k][2] < 0):
                
                print('Densidad < 0. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count1 = count1 + 1
            else:
                pass
            
            if any(ListaModelos[i][j][k][3] < 0):
                
                print('Presión radial < 0. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count2 = count2 + 1
            else:
                pass
            
            if any(ListaModelos[i][j][k][4] < 0):
                
                print('Presión tangencial < 0. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count3 = count3 + 1
            else:
                continue
                
print('Número total de modelos que no cumplen Densidad > 0:           ', count1, ' de ', Number_of_models)
print('Número total de modelos que no cumplen Presión radial > 0:     ', count2, ' de ', Number_of_models)
print('Número total de modelos que no cumplen Presión tangencial > 0: ', count3, ' de ', Number_of_models)

# <center> Density gradient, Radial Pressure gradient and Tangetial Pressure gradient  < 0
# <center> C5

In [ ]:
count1 = 0
count2 = 0
count3 = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if any(ListaModelos[i][j][k][5][:] > 0):
                
                print('Gradiente de densidad > 0. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count1 = count1 + 1
            else:
                pass
            
            if any(ListaModelos[i][j][k][6][:] > 0):
                
                print('Gradiente de presión > 0. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count2 = count2 + 1
            else:
                pass
            
            if any(ListaModelos[i][j][k][7][round(len(ListaModelos[i][j][k][7])*0.1):] > 0):
                
                print('Gradiente de presión tangencial > 0. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count3 = count3 + 1
            else:
                continue

print('Número total de modelos que no cumplen Gradiende de Densidad < 0:           ', count1, ' de ',Number_of_models)
print('Número total de modelos que no cumplen Gradiente de Presión radial < 0:     ', count2, ' de ',Number_of_models)
print('Número total de modelos que no cumplen Gradiente de Presión tangencial < 0: ', count3, ' de ',Number_of_models)

# <center> Strong Energy Condition
# <center> C6

In [ ]:
count = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if any(ListaModelos[i][j][k][8] < 0):
                
                print('No cumple la condición de energía fuerte (SEC). Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count = count + 1
            else:
                continue

print('Número total de modelos que no cumplen: ', count, ' de ',len(Lista_C)*len(Lista_n)*len(Lista_sigma))

# <center> Radial and Tangential speed of sound squared
# <center> C7

In [ ]:
count1 = 0
count2 = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if any(ListaModelos[i][j][k][9] > 1):
                print('Velocidad del sonido > 1. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count1 = count1 + 1
            else:
                pass
            
            if any(ListaModelos[i][j][k][10] > 1):                
                print('Velocidad del sonido tangencial > 1. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count2 = count2 + 1
            else:
                continue
                
print('Número total de modelos que no cumplen: ', count1, ' de ',len(Lista_C)*len(Lista_n)*len(Lista_sigma))
print('Número total de modelos que no cumplen: ', count2, ' de ',len(Lista_C)*len(Lista_n)*len(Lista_sigma))

# <center> Adiabatic index Gamma
# <center> C8

In [ ]:
count = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if any(ListaModelos[i][j][k][11] < 4/3):
                print('Gamma < 4/3. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count = count + 1
            else:
                continue
                
print('Número total de modelos que no cumplen: ', count, ' de ',len(Lista_C)*len(Lista_n)*len(Lista_sigma))

# <center> Cracking
# <center> C9

In [ ]:
count = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if any(np.diff(np.sign(ListaModelos[i][j][k][12][100:])) != 0):
                print('El modelo fractura. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count = count + 1
            else:
                continue
                
print('Número total de modelos que no cumplen: ', count, ' de ',len(Lista_C)*len(Lista_n)*len(Lista_sigma))

# <center> Convective Stability
# <center> C11

In [ ]:
count = 0

for i in range(len(Lista_C)):
    
    for j in range(len(Lista_n)):
        
        for k in range(len(Lista_sigma)):
            
            if any(ListaModelos[i][j][k][13] > 0):
                print('Inestable ante perturbaciones convectivas. Parámetros: ','C = %.3f, '%(Lista_C[i]),
                     'n = %.1f, '%(Lista_n[j]),'\u03c3 = %.2f, '%(Lista_sigma[k]))
                count = count + 1                
            else:
                continue
                
print('Número total de modelos que no cumplen: ', count, ' de ',len(Lista_C)*len(Lista_n)*len(Lista_sigma))